In [3]:
# Importing the libraries
import urllib.request
import zipfile

In [4]:
# Details
url = 'https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip'
name = 'data.zip'
train_dir = 'data/train/'
urllib.request.urlretrieve(url, name)

In [3]:
# Unzipping
zipped = zipfile.ZipFile(name, 'r')
zipped.extractall(train_dir)
zipped.close()

In [5]:
# Image generator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#Scale by 1/255
train_datagen = ImageDataGenerator(rescale=1/255)

# Flow from directory
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(300, 300),
    class_mode='binary'
)

Found 1027 images belonging to 2 classes.


In [21]:
# Image augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#Scale by 1/255
train_datagen = ImageDataGenerator(
    rescale=1/255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
    )

# Flow from directory
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(300, 300),
    class_mode='binary'
)

Found 1027 images belonging to 2 classes.


In [22]:
# CNN architecture
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = tf.keras.models.Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(300, 300, 3)),
    MaxPooling2D(2, 2),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [23]:
# Summary
# 1.7 mil parameters 🤯
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 298, 298, 16)      448       
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 149, 149, 16)     0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 147, 147, 32)      4640      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 73, 73, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 71, 71, 64)        18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 35, 35, 64)      

In [24]:
# Compiling the model
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy', optimizer=RMSprop(learning_rate=0.001), metrics=['accuracy'])

In [14]:
# Training the model
history = model.fit_generator(
    train_generator,
    epochs=15,
)

/var/folders/xp/g_y_0gnd0v37gfx63299g9_00000gn/T/ipykernel_38681/2970098988.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/15
33/33 [==============================] - 6s 139ms/step - loss: 0.4605 - accuracy: 0.8218
Epoch 2/15
33/33 [==============================] - 4s 130ms/step - loss: 0.2037 - accuracy: 0.9231
Epoch 3/15
33/33 [==============================] - 4s 130ms/step - loss: 0.1163 - accuracy: 0.9581
Epoch 4/15
33/33 [==============================] - 4s 130ms/step - loss: 0.2583 - accuracy: 0.9698
Epoch 5/15
33/33 [==============================] - 4s 133ms/step - loss: 0.1104 - accuracy: 0.9718
Epoch 6/15
33/33 [==============================] - 4s 130ms/step - loss: 0.7680 - accuracy: 0.9776
Epoch 7/15
33/33 [==============================] - 4s 129ms/step - loss: 0.0267 - accuracy: 0.9893
Epoch 8/15
33/33 [==============================] - 4s 130ms/step - loss: 9.5717e-04 - accuracy: 1.0000
Epoch 9/15
33/33 [==============================] - 4s 131ms/step - loss: 9.4963e-05 - accuracy: 1.0000
Epoch 10/15
33/33 [==============================] - 4s 130ms/step - loss: 0.2038 - accuracy

In [9]:
# Details
url = 'https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip'
name = 'valid-data.zip'
valid = 'data/validation/'
urllib.request.urlretrieve(url, name)

In [17]:
# Unzipping
zipped = zipfile.ZipFile(name, 'r')
zipped.extractall(train_dir)
zipped.close()

In [25]:
# Validation Image generator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#Scale by 1/255
valid_datagen = ImageDataGenerator(rescale=1/255)

# Flow from directory
valid_generator = train_datagen.flow_from_directory(
    valid,
    target_size=(300, 300),
    class_mode='binary'
)

Found 256 images belonging to 2 classes.


In [26]:
# Training the model
history = model.fit_generator(
    train_generator,
    epochs=15,
    validation_data=valid_generator,
)

/var/folders/xp/g_y_0gnd0v37gfx63299g9_00000gn/T/ipykernel_40878/2718311792.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/15


2022-05-30 10:12:48.459400: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - ETA: 0s - loss: 0.7818 - accuracy: 0.5219

2022-05-30 10:13:02.060756: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 17s 486ms/step - loss: 0.7818 - accuracy: 0.5219 - val_loss: 0.6849 - val_accuracy: 0.5117
Epoch 2/15
33/33 [==============================] - 16s 480ms/step - loss: 0.6353 - accuracy: 0.6349 - val_loss: 1.2688 - val_accuracy: 0.5586
Epoch 3/15
33/33 [==============================] - 16s 485ms/step - loss: 0.5503 - accuracy: 0.7313 - val_loss: 0.8852 - val_accuracy: 0.6055
Epoch 4/15
33/33 [==============================] - 16s 476ms/step - loss: 0.4461 - accuracy: 0.7936 - val_loss: 0.5252 - val_accuracy: 0.7578
Epoch 5/15
33/33 [==============================] - 15s 469ms/step - loss: 0.3664 - accuracy: 0.8403 - val_loss: 1.3074 - val_accuracy: 0.5977
Epoch 6/15
33/33 [==============================] - 15s 464ms/step - loss: 0.3288 - accuracy: 0.8793 - val_loss: 0.4372 - val_accuracy: 0.8281
Epoch 7/15
33/33 [==============================] - 16s 481ms/step - loss: 0.1845 - accuracy: 0.9202 - val_loss: 2.1651 - val_accuracy: 0.6250
Epoch 8/15

In [14]:
import numpy as np
from keras.preprocessing import image

def predict_image(image_path):
    img = image.load_img(image_path, target_size=(300, 300))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)

    image_tensor = np.vstack([x])
    classes = model.predict(image_tensor)
    print(classes)
    print(classes[0])

    if classes[0] > 0.5:
        print('Prediction: Human')
    else:
        print('Prediction: Horse')